In [ ]:
from elasticsearch import Elasticsearch

client = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic", "Q8bleCO5jiX1y=-0Pk*L"),
    verify_certs=False
)

In [103]:
client.info()
client.cat.indices(s='index', format='json')

c:\Users\Alma\Desktop\TerraWatt\Limpieza_datos\.conda\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\Alma\Desktop\TerraWatt\Limpieza_datos\.conda\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ListApiResponse([{'health': 'yellow', 'status': 'open', 'index': 'modelo_consumo_met_fest', 'uuid': 'OmbVE--uTxyVpPW4sZxpVQ', 'pri': '1', 'rep': '1', 'docs.count': '18300', 'docs.deleted': '0', 'store.size': '3.7mb', 'pri.store.size': '3.7mb', 'dataset.size': '3.7mb'}, {'health': 'yellow', 'status': 'open', 'index': 'modelo_consumo_met_fest_python', 'uuid': 'KdV6e2CSR62G6HH4fnLWiQ', 'pri': '1', 'rep': '1', 'docs.count': '0', 'docs.deleted': '0', 'store.size': '249b', 'pri.store.size': '249b', 'dataset.size': '249b'}, {'health': 'yellow', 'status': 'open', 'index': 'modelo_precios_met_fest_python', 'uuid': 'BMfpGz4KRvCp7pvf65c40g', 'pri': '1', 'rep': '1', 'docs.count': '193500', 'docs.deleted': '0', 'store.size': '38.8mb', 'pri.store.size': '38.8mb', 'dataset.size': '38.8mb'}])

In [100]:
list(client.cat.indices(s='index', format='json'))

c:\Users\Alma\Desktop\TerraWatt\Limpieza_datos\.conda\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[{'health': 'yellow',
  'status': 'open',
  'index': 'modelo_consumo_met_fest',
  'uuid': 'OmbVE--uTxyVpPW4sZxpVQ',
  'pri': '1',
  'rep': '1',
  'docs.count': '18300',
  'docs.deleted': '0',
  'store.size': '3.7mb',
  'pri.store.size': '3.7mb',
  'dataset.size': '3.7mb'},
 {'health': 'yellow',
  'status': 'open',
  'index': 'modelo_consumo_met_fest_python',
  'uuid': 'KdV6e2CSR62G6HH4fnLWiQ',
  'pri': '1',
  'rep': '1',
  'docs.count': '0',
  'docs.deleted': '0',
  'store.size': '249b',
  'pri.store.size': '249b',
  'dataset.size': '249b'},
 {'health': 'yellow',
  'status': 'open',
  'index': 'modelo_precios_met_fest_python',
  'uuid': 'BMfpGz4KRvCp7pvf65c40g',
  'pri': '1',
  'rep': '1',
  'docs.count': '193500',
  'docs.deleted': '0',
  'store.size': '38.8mb',
  'pri.store.size': '38.8mb',
  'dataset.size': '38.8mb'}]

In [101]:
# Consultar los índices existentes
existing_indices = client.cat.indices(format="json")
index_names = [index["index"] for index in existing_indices]

# Crear un nuevo índice solo si no existe
new_index_name = "modelo_consumo_met_fest_python"
if new_index_name not in index_names:
    client.indices.create(index=new_index_name)
    print(f"Índice '{new_index_name}' creado exitosamente.")
else:
    print(f"El índice '{new_index_name}' ya existe.")


El índice 'modelo_consumo_met_fest_python' ya existe.


c:\Users\Alma\Desktop\TerraWatt\Limpieza_datos\.conda\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [90]:
import pandas as pd
from elasticsearch.helpers import bulk

# Cargar el archivo CSV
df = pd.read_csv("./Modelo_Consumo_Met_fest.csv", delimiter=';')

# Crear una lista de documentos para el bulk
actions = []

for index, row in df.iterrows():
    # Crear cada documento en el formato esperado por Elasticsearch
    doc = {
        "_index": "modelo_consumo_met_fest", 
        "_id": f"modelo_consumo_{index}", 
        "_source": {
            "Fecha": row["Fecha"],
            "Provincia": row["Provincia"],
            "Consumo_energetico": row["Consumo energético (kWh/m²)"],
            "Media_residentes": row["Media de residentes"],
            "Potencia_contratada": row["Potencia contratada (kW)"],
            "Tipo_vivienda": row["Tipo de vivienda"],
            "Altitud": row["ALTITUD"],
            "TMedia": row["TMEDIA"],
            "TMin": row["TMIN"],
            "TMax": row["TMAX"],
            "Dir": row["DIR"],
            "VelMedia": row["VELMEDIA"],
            "Racha": row["RACHA"],
            "Sol": row["SOL"],
            "PresMax": row["PRESMAX"],
            "PresMin": row["PRESMIN"],
            "Festivo": row["Festivo"],
            "Entre_semana": row["Entre semana"]
        }
    }
    # Añadir el documento a la lista de acciones
    actions.append(doc)

    # Mostrar progreso cada 500 documentos
    if index % 500 == 0:
        print(f"Preparados {index} documentos para el bulk")

# Enviar todos los documentos a Elasticsearch usando el bulk
response = bulk(client, actions)
print(f"Bulk completado. {len(actions)} documentos indexados.")


Preparados 0 documentos para el bulk
Preparados 500 documentos para el bulk
Preparados 1000 documentos para el bulk
Preparados 1500 documentos para el bulk
Preparados 2000 documentos para el bulk
Preparados 2500 documentos para el bulk
Preparados 3000 documentos para el bulk
Preparados 3500 documentos para el bulk
Preparados 4000 documentos para el bulk
Preparados 4500 documentos para el bulk
Preparados 5000 documentos para el bulk
Preparados 5500 documentos para el bulk
Preparados 6000 documentos para el bulk
Preparados 6500 documentos para el bulk
Preparados 7000 documentos para el bulk
Preparados 7500 documentos para el bulk
Preparados 8000 documentos para el bulk
Preparados 8500 documentos para el bulk
Preparados 9000 documentos para el bulk
Preparados 9500 documentos para el bulk
Preparados 10000 documentos para el bulk
Preparados 10500 documentos para el bulk
Preparados 11000 documentos para el bulk
Preparados 11500 documentos para el bulk
Preparados 12000 documentos para el bulk

c:\Users\Alma\Desktop\TerraWatt\Limpieza_datos\.conda\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\Alma\Desktop\TerraWatt\Limpieza_datos\.conda\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\Alma\Desktop\TerraWatt\Limpieza_datos\.conda\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warn

Bulk completado. 18300 documentos indexados.


In [91]:
# Consultar los índices existentes
existing_indices = client.cat.indices(format="json")
index_names = [index["index"] for index in existing_indices]

# Crear un nuevo índice solo si no existe
new_index_name = "modelo_precios_met_fest_python"
if new_index_name not in index_names:
    client.indices.create(index=new_index_name)
    print(f"Índice '{new_index_name}' creado exitosamente.")
else:
    print(f"El índice '{new_index_name}' ya existe.")


c:\Users\Alma\Desktop\TerraWatt\Limpieza_datos\.conda\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\Alma\Desktop\TerraWatt\Limpieza_datos\.conda\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Índice 'modelo_precios_met_fest_python' creado exitosamente.


In [92]:
import pandas as pd
from elasticsearch.helpers import bulk

# Cargar el archivo CSV
df = pd.read_csv("./Modelo_Precios_Met_Fest.csv", delimiter=';')

# Crear una lista de documentos para el bulk
actions = []

for index, row in df.iterrows():
    # Crear cada documento en el formato esperado por Elasticsearch
    doc = {
        "_index": "modelo_precios_met_fest_python",  # Nombre del índice
        "_id": f"modelo_precios_{index}",  # ID único por documento
        "_source": {
            "Fecha": row["FECHA"],
            "Provincia": row["Provincia"],
            "Altitud": row["ALTITUD"],
            "TMedia": row["TMEDIA"],
            "TMin": row["TMIN"],
            "TMax": row["TMAX"],
            "Dir": row["DIR"],
            "VelMedia": row["VELMEDIA"],
            "Racha": row["RACHA"],
            "Sol": row["SOL"],
            "PresMax": row["PRESMAX"],
            "PresMin": row["PRESMIN"],
            "Precio_total_con_impuestos": row["Precio total con impuestos (€/MWh)"],
            "Festivo": row["Festivo"],
            "Entre_semana": row["Entre semana"]
        }
    }
    # Añadir el documento a la lista de acciones
    actions.append(doc)

    # Mostrar progreso cada 500 documentos
    if index % 500 == 0:
        print(f"Preparados {index} documentos para el bulk")

# Enviar todos los documentos a Elasticsearch usando el bulk
response = bulk(client, actions)
print(f"Bulk completado. {len(actions)} documentos indexados.")


Preparados 0 documentos para el bulk
Preparados 500 documentos para el bulk
Preparados 1000 documentos para el bulk
Preparados 1500 documentos para el bulk
Preparados 2000 documentos para el bulk
Preparados 2500 documentos para el bulk
Preparados 3000 documentos para el bulk
Preparados 3500 documentos para el bulk
Preparados 4000 documentos para el bulk
Preparados 4500 documentos para el bulk
Preparados 5000 documentos para el bulk
Preparados 5500 documentos para el bulk
Preparados 6000 documentos para el bulk
Preparados 6500 documentos para el bulk
Preparados 7000 documentos para el bulk
Preparados 7500 documentos para el bulk
Preparados 8000 documentos para el bulk
Preparados 8500 documentos para el bulk
Preparados 9000 documentos para el bulk
Preparados 9500 documentos para el bulk
Preparados 10000 documentos para el bulk
Preparados 10500 documentos para el bulk
Preparados 11000 documentos para el bulk
Preparados 11500 documentos para el bulk
Preparados 12000 documentos para el bulk

c:\Users\Alma\Desktop\TerraWatt\Limpieza_datos\.conda\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\Alma\Desktop\TerraWatt\Limpieza_datos\.conda\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\Alma\Desktop\TerraWatt\Limpieza_datos\.conda\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warn

Bulk completado. 193500 documentos indexados.


c:\Users\Alma\Desktop\TerraWatt\Limpieza_datos\.conda\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Para comprobar que esta bien cargado realizaremos una consulta de prueba

In [ ]:
# # Consultar los primeros 10 documentos del índice
# search_response = client.search(
#     index=new_index_name,
#     body={"query": {"match_all": {}}, "size": 10}
# )
# for doc in search_response["hits"]["hits"]:
#     print(doc["_source"])


c:\Users\Alma\Desktop\TerraWatt\Limpieza_datos\.conda\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'Fecha': '2014-04-01', 'Provincia': 'LLEIDA', 'Altitud': 748.9, 'TMedia': 11.52, 'TMin': 4.96, 'TMax': 18.075, 'Dir': 24.0, 'VelMedia': 2.0083333333333333, 'Racha': 9.6, 'Sol': 8.75, 'PresMax': 956.95, 'PresMin': 951.75, 'Precio_total_con_impuestos': 31.57719416140608, 'Festivo': 'NO', 'Entre_semana': 'SI'}
{'Fecha': '2014-04-02', 'Provincia': 'LLEIDA', 'Altitud': 745.4761904761905, 'TMedia': 10.752380952380951, 'TMin': 5.3, 'TMax': 16.228571428571428, 'Dir': 22.666666666666668, 'VelMedia': 2.708333333333333, 'Racha': 12.875, 'Sol': 0.75, 'PresMax': 953.0, 'PresMin': 943.15, 'Precio_total_con_impuestos': 49.26139798723967, 'Festivo': 'NO', 'Entre_semana': 'SI'}
{'Fecha': '2014-04-03', 'Provincia': 'LLEIDA', 'Altitud': 745.4761904761905, 'TMedia': 8.84, 'TMin': 6.17, 'TMax': 11.49, 'Dir': 17.818181818181817, 'VelMedia': 1.5916666666666668, 'Racha': 10.627272727272729, 'Sol': 0.0, 'PresMax': 944.9, 'PresMin': 939.55, 'Precio_total_con_impuestos': 44.02608933889056, 'Festivo': 'NO', 'Ent